# 1. CARGO DATAFRAMES BASE

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

ruta_export = r"C:\Users\MSII7\Documents\Jasef\Desercion\export"
ruta_output = r"C:\Users\MSII7\Documents\Jasef\Desercion\output"
ruta_sources = r"C:\Users\MSII7\Documents\Jasef\Desercion\sources"

df_matriculados_2024_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2024_1.csv'))
df_matriculados_2023_2 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2023_2.csv'))
df_matriculados_2023_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2023_1.csv'))
df_matriculados_2022_2 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2022_2.csv'))
df_matriculados_2022_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2022_1.csv'))


df_desertores_2023_2 = pd.read_csv(os.path.join(ruta_export,'Desertores_2023_2.csv'))

df_variables = pd.read_csv(os.path.join(ruta_export,'DataMaestra_Estudiante.csv'))

df_notas_2023_2 = pd.read_csv(os.path.join(ruta_sources,'DATA_DETALLE_NOTAS-2023-2.csv'))

df_colegio_procedencia = pd.read_csv(
    os.path.join(ruta_sources, 'DATA_COLEGIO_PROCEDENCIA.csv'),
    header=None,  # Indica que el archivo no tiene cabeceras
    names=['IdAlumno', 'Colegio', 'TipoColegio']  # Especificar las cabeceras manualmente
)

nota_aprobatoria = 13
notas_desaprobadas = 9
periodo = '2023-2'

# OBTENGO VARIABLES RELACIONADAS A NOTAS
### Parciales Desaprobados
### Finales Desaprobados
### Flag Excede Notas Desaprobadas No Parciales No Finales
### Flag que indica si la cantidad de cursos desaprobados excede o es igual al 50% del total de cursos

In [2]:
# Agrupar por IdAlumno y contar los registros de parciales y finales desaprobados
df_cant_desaprobado_agrupado = df_notas_2023_2.groupby('IdAlumno').agg(
    CantParcialesDesaprobados=(
        'Actividad', 
        lambda x: ((x == 'EVALUACION PARCIAL') & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < nota_aprobatoria)).sum()
    ),
    CantFinalesDesaprobados=(
        'Actividad', 
        lambda x: ((x == 'EVALUACION FINAL') & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < nota_aprobatoria)).sum()
    ),
    # Columna auxiliar para contar las evaluaciones diferentes de PARCIAL y FINAL desaprobadas
    CantNotasDesaprobadasNoParcNoFin=(
        'Actividad', 
        lambda x: ((~x.isin(['EVALUACION PARCIAL', 'EVALUACION FINAL', 'EVALUACION DIAGNOSTICA'])) & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < nota_aprobatoria)).sum()
    )
).reset_index()

# Crear la columna 'ExcedeNotasDesaprobadasNoParcNoFin'
df_cant_desaprobado_agrupado['ExcedeNotasDesaprobadasNoParcNoFin'] = df_cant_desaprobado_agrupado['CantNotasDesaprobadasNoParcNoFin'].apply(
    lambda x: 'SI' if x >= notas_desaprobadas else 'NO'
)

# Eliminar la columna 'CantNotasDesaprobadasNoParcNoFin' 
df_cant_desaprobado_agrupado.drop(columns=['CantNotasDesaprobadasNoParcNoFin'], inplace=True)

# Calcular la nota final para cada curso por alumno
df_notas_2023_2['NotaFinalCurso'] = np.where(
    df_notas_2023_2['Actividad'] != 'EVALUACION DIAGNOSTICA',
    (df_notas_2023_2['Peso'] * df_notas_2023_2['NotaActiv']) / 100,
    np.nan  # Ignoramos las actividades diagnósticas
)

# Agrupar por IdAlumno y Curso para obtener la nota final por curso
df_notas_finales_por_curso = df_notas_2023_2.groupby(['IdAlumno', 'Curso']).agg(
    NotaFinalCurso=('NotaFinalCurso', 'sum')  # Sumar las notas ponderadas por curso
).reset_index()

# Redondeo las notas a su entero más cercano para el posterior conteo 
df_notas_finales_por_curso['NotaFinalCurso'] = df_notas_finales_por_curso['NotaFinalCurso'].round()

# Agrupar por IdAlumno y contar los cursos desaprobados
df_cursos_desaprobados = df_notas_finales_por_curso.groupby('IdAlumno').agg(
    CantCursosDesaprobados=(
        'NotaFinalCurso', 
        lambda x: (x < nota_aprobatoria).sum()
    ),
    CantCursos=(
        'NotaFinalCurso', 
        'count'  # Cuenta la cantidad de cursos
    )
).reset_index()

# Creamos la columna ExcedeMitadCursosDesaprobados
df_cursos_desaprobados['ExcedeMitadCursosDesaprobados'] = df_cursos_desaprobados.apply(
    lambda row: 'SI' if (row['CantCursosDesaprobados'] / row['CantCursos']) >= 0.5 else 'NO', axis=1
)

# Eliminar la columna 'CantNotasDesaprobadasNoParcNoFin' 
df_cursos_desaprobados.drop(columns=['CantCursosDesaprobados','CantCursos'], inplace=True)

# Unir este resultado con el DataFrame anterior
df_cant_desaprobado_agrupado = df_cant_desaprobado_agrupado.merge(
    df_cursos_desaprobados, 
    on='IdAlumno', 
    how='left'
)

df_cant_desaprobado_agrupado = df_cant_desaprobado_agrupado.merge(
    df_notas_2023_2[['IdAlumno', 'ProgramaAlu']].drop_duplicates(),  # Tomar IdAlumno y ProgramaAlu sin duplicados
    on='IdAlumno',
    how='left'
)

# Visualizar el DataFrame final
print(df_cant_desaprobado_agrupado.head())

    IdAlumno  CantParcialesDesaprobados  CantFinalesDesaprobados  \
0  100002894                          0                        0   
1  100003558                          3                        3   
2  100003932                          1                        0   
3  100003946                          2                        3   
4  100003989                          0                        0   

  ExcedeNotasDesaprobadasNoParcNoFin ExcedeMitadCursosDesaprobados  \
0                                 NO                            NO   
1                                 NO                            SI   
2                                 NO                            NO   
3                                 NO                            NO   
4                                 NO                            NO   

                     ProgramaAlu  
0          Nutrición y Dietética  
1  Medic Veterinaria y Zootecnia  
2                Medicina Humana  
3  Medic Veterinaria y Zootecn

# 2. Identificación de alumnos a predecir

In [3]:
# Lista de DataFrames a combinar dinámicamente
dfs_matriculados = [df_matriculados_2023_1, df_matriculados_2022_2, df_matriculados_2022_1, df_matriculados_2023_2]

# Concatenar todos los DataFrames
df_matriculados_todos = pd.concat(dfs_matriculados)

# Agrupar por 'IdAlumno' y obtener el mínimo valor de 'CicloLectivoDesCorta'
df_matriculados_agrupados = df_matriculados_todos.groupby(['IdAlumno']).agg({'CicloLectivoDesCorta': 'min'}).reset_index()


# Filtrar los alumnos cuyo mínimo 'CicloLectivoDesCorta' sea igual al valor definido
df_nuevos_matriculados = df_matriculados_agrupados[df_matriculados_agrupados['CicloLectivoDesCorta'] == periodo]

# Mostrar el DataFrame resultante
print(df_nuevos_matriculados)


         IdAlumno CicloLectivoDesCorta
14      100004070               2023-2
46      100008630               2023-2
48      100009785               2023-2
62      100011191               2023-2
86      100012563               2023-2
...           ...                  ...
20656   180000324               2023-2
20673   180000558               2023-2
20745   180000790               2023-2
21058  4200710473               2023-2
21061  4200810288               2023-2

[1205 rows x 2 columns]


In [4]:
# Realizar el cruce con el DataFrame de desertores
df_nuevos_matriculados = df_nuevos_matriculados.merge(
    df_desertores_2023_2[['IdAlumno']], 
    on='IdAlumno', 
    how='left', 
    indicator=True
)

# Crear la columna 'Desercion' donde se marca con 1 si está en df_desertores_2023_2 ('both') y 0 si no está ('left_only')
df_nuevos_matriculados['Desercion'] = df_nuevos_matriculados['_merge'].apply(lambda x: 1 if x == 'both' else 0)

# Eliminar la columna '_merge' que se creó con el indicador
df_nuevos_matriculados.drop(columns=['_merge'], inplace=True)

# Mostrar el DataFrame resultante con la nueva columna 'Desercion'
print(df_nuevos_matriculados)

        IdAlumno CicloLectivoDesCorta  Desercion
0      100004070               2023-2          0
1      100008630               2023-2          0
2      100009785               2023-2          0
3      100011191               2023-2          1
4      100012563               2023-2          1
...          ...                  ...        ...
1200   180000324               2023-2          1
1201   180000558               2023-2          0
1202   180000790               2023-2          0
1203  4200710473               2023-2          1
1204  4200810288               2023-2          1

[1205 rows x 3 columns]


### Validacion de alumno que no debe aparecer para 2023-2

In [5]:
# Filtrar el DataFrame para encontrar el registro específico
resultado = df_nuevos_matriculados[df_nuevos_matriculados['IdAlumno'] == 100118394]

# Verifica el resultado
print(resultado)

Empty DataFrame
Columns: [IdAlumno, CicloLectivoDesCorta, Desercion]
Index: []


# CONSTRUYO UN SOLO DATAFRAME CON TODAS LAS VARIABLES

In [6]:
#Variables CRM
df = pd.merge(df_nuevos_matriculados, df_variables, on='IdAlumno', how='left')

#Parciales Desaprobados
df = pd.merge(df, df_cant_desaprobado_agrupado, on='IdAlumno', how='left')

#Colegio Procedencia
df = pd.merge(df,df_colegio_procedencia, on='IdAlumno', how='left')


# columna objetivo
df['Desercion'] = df['IdAlumno'].isin(df_desertores_2023_2['IdAlumno']).astype(int)

print(df.head())

    IdAlumno CicloLectivoDesCorta  Desercion Genero FechaNacimiento Distrito  \
0  100004070               2023-2          0    NaN             NaN      NaN   
1  100008630               2023-2          0    NaN             NaN      NaN   
2  100009785               2023-2          0    NaN             NaN      NaN   
3  100011191               2023-2          1    NaN             NaN      NaN   
4  100012563               2023-2          1    NaN             NaN      NaN   

  Provincia Departamento FamiliarResponsable  AnioNacimiento  \
0       NaN          NaN                 NaN             NaN   
1       NaN          NaN                 NaN             NaN   
2       NaN          NaN                 NaN             NaN   
3       NaN          NaN                 NaN             NaN   
4       NaN          NaN                 NaN             NaN   

   CantParcialesDesaprobados  CantFinalesDesaprobados  \
0                          0                        0   
1                   

In [7]:
print(df.shape)
df.to_csv('cachimbos_2023_2_nulos.csv',index=False)

(1211, 17)


In [8]:
print(df.isnull().sum())

IdAlumno                                0
CicloLectivoDesCorta                    0
Desercion                               0
Genero                                 90
FechaNacimiento                        90
Distrito                               90
Provincia                              90
Departamento                           90
FamiliarResponsable                   745
AnioNacimiento                         90
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
dtype: int64


### RELLENO LOS QUE NO TIENEN GENERO

In [9]:
# Calculamos la cuenta de cada género y de los valores nulos
conteo_generos = df['Genero'].value_counts(dropna=False)
count_f = conteo_generos.get('F', 0)  # Cantidad de F
count_m = conteo_generos.get('M', 0)  # Cantidad de M
total_blancos = conteo_generos.get(np.nan, 0)  # Cantidad de valores nulos

# Ahora, calculamos la distribución equitativa
if total_blancos > 0:
    # Proporciones de cada género
    proporciones_f = count_f / (count_f + count_m)
    proporciones_m = count_m / (count_f + count_m)

    # Cálculo de cuántos géneros se asignarán a los valores nulos
    asignacion_f = int(total_blancos * proporciones_f)
    asignacion_m = total_blancos - asignacion_f  # Lo que queda se asigna a M

    # Rellenar los valores nulos en el DataFrame
    df.loc[df['Genero'].isna(), 'Genero'] = ['F'] * asignacion_f + ['M'] * asignacion_m


print(df.isnull().sum())


IdAlumno                                0
CicloLectivoDesCorta                    0
Desercion                               0
Genero                                  0
FechaNacimiento                        90
Distrito                               90
Provincia                              90
Departamento                           90
FamiliarResponsable                   745
AnioNacimiento                         90
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
dtype: int64


### RELLENO LOS QUE NO TIENEN FECHA DE NACIMIENTO

In [10]:
# Contar cuántos valores nulos hay en FechaNacimiento
n = df['FechaNacimiento'].isnull().sum()

# Función para generar fechas aleatorias en el formato 'YYYY-MM-DD'
def generar_fecha_aleatoria(n):
    fechas = []
    for _ in range(n):
        anio = np.random.choice([2006, 2007])  # Años para tener 16 o 17 años en 2023
        mes = np.random.randint(1, 13)  # Mes de 1 a 12
        dia = np.random.randint(1, 29)  # Día de 1 a 28 (para simplificar)
        fecha = f"{anio}-{mes:02d}-{dia:02d}"  # Formato 'YYYY-MM-DD'
        fechas.append(fecha)
    return fechas

# Generar fechas aleatorias y rellenar los nulos
fechas_aleatorias = generar_fecha_aleatoria(n)
df.loc[df['FechaNacimiento'].isnull(), 'FechaNacimiento'] = fechas_aleatorias

# Obtener el año de nacimiento de los nulos en FechaNacimiento
df.loc[df['AnioNacimiento'].isnull(), 'AnioNacimiento'] = df['FechaNacimiento'].str.split('-').str[0].astype(int)

#print(df)
print(df.isnull().sum())

IdAlumno                                0
CicloLectivoDesCorta                    0
Desercion                               0
Genero                                  0
FechaNacimiento                         0
Distrito                               90
Provincia                              90
Departamento                           90
FamiliarResponsable                   745
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
dtype: int64


### RELLENO LOS QUE NO TIENEN DISTRITO

In [11]:
# Lista de distritos para rellenar
distritos = ["ATE", "VILLA EL SALVADOR", "COMAS", "CHORRILLOS", "SAN MARTÍN DE PORRES"]

# Contar cuántos valores nulos hay en el campo 'Distrito'
nulos_distrito = df['Distrito'].isnull().sum()

# Calcular cuántos registros de cada distrito se necesitan
if nulos_distrito > 0:
    # Calcular cuántos distritos asignar a cada uno
    asignaciones = [nulos_distrito // len(distritos)] * len(distritos)
    
    # Distribuir los restantes de manera aleatoria entre los distritos
    for i in range(nulos_distrito % len(distritos)):
        asignaciones[i] += 1

    # Crear una lista con los distritos asignados
    distritos_asignados = []
    for distrito, cantidad in zip(distritos, asignaciones):
        distritos_asignados.extend([distrito] * cantidad)

    # Mezclar aleatoriamente la lista para distribuir uniformemente
    np.random.shuffle(distritos_asignados)

    # Rellenar los valores nulos en el DataFrame
    df.loc[df['Distrito'].isnull(), 'Distrito'] = distritos_asignados

# Mostrar el DataFrame resultante
print(df.isnull().sum())

IdAlumno                                0
CicloLectivoDesCorta                    0
Desercion                               0
Genero                                  0
FechaNacimiento                         0
Distrito                                0
Provincia                              90
Departamento                           90
FamiliarResponsable                   745
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
dtype: int64


### RELLENO LOS QUE NO TIENEN PROVINCIA NI DEPARTAMENTO

In [12]:
df['Provincia'] = df['Provincia'].fillna('LIMA')
df['Departamento'] = df['Departamento'].fillna('LIMA')

print(df.isnull().sum())

IdAlumno                                0
CicloLectivoDesCorta                    0
Desercion                               0
Genero                                  0
FechaNacimiento                         0
Distrito                                0
Provincia                               0
Departamento                            0
FamiliarResponsable                   745
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
dtype: int64


### CREO CAMPO ANIO

In [13]:
# Crear la columna 'ANIO' tomando los primeros 4 caracteres de 'CicloLectivoDesCorta'
df['Anio'] = df['CicloLectivoDesCorta'].str[:4].astype(int)

### CREO CAMPO EDAD

In [14]:
df['Edad'] = df['Anio'] - df['AnioNacimiento']
print(df.isnull().sum())

IdAlumno                                0
CicloLectivoDesCorta                    0
Desercion                               0
Genero                                  0
FechaNacimiento                         0
Distrito                                0
Provincia                               0
Departamento                            0
FamiliarResponsable                   745
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
Anio                                    0
Edad                                    0
dtype: int64


In [15]:
# Definir las condiciones y sus correspondientes etiquetas
condiciones = [
    (df['Edad'] <= 18),
    (df['Edad'] > 18) & (df['Edad'] < 25),
    (df['Edad'] >= 25)
]

etiquetas = ['MenosDe18', 'Entre18y25', 'Mayor25']

# Crear la nueva columna 'GrupoEdad' con las etiquetas según las condiciones, usando un valor por defecto como ''
df['GrupoEdad'] = np.select(condiciones, etiquetas, default='')

# Verificar los resultados
print(df[['Edad', 'GrupoEdad']].head())

   Edad  GrupoEdad
0  17.0  MenosDe18
1  16.0  MenosDe18
2  16.0  MenosDe18
3  17.0  MenosDe18
4  17.0  MenosDe18


### RELLENO EL FAMILIAR RESPONSABLE

In [16]:
# Filas donde FamiliarResponsable es nulo
mask = df['FamiliarResponsable'].isnull()

# Aplicar la condición solo a esas filas
df.loc[mask, 'FamiliarResponsable'] = df.loc[mask, 'Edad'].apply(lambda x: 'Apoderado' if x <= 18 else 'Alumno')

### RELLENO EL TIPO DE COLEGIO

In [17]:
df['TipoColegio'] = df_colegio_procedencia['TipoColegio'].replace({'RELG': 'PRIV', 'OTHR': 'PUBL'}).fillna('PUBL')

In [18]:
print(df.isnull().sum())

IdAlumno                              0
CicloLectivoDesCorta                  0
Desercion                             0
Genero                                0
FechaNacimiento                       0
Distrito                              0
Provincia                             0
Departamento                          0
FamiliarResponsable                   0
AnioNacimiento                        0
CantParcialesDesaprobados             0
CantFinalesDesaprobados               0
ExcedeNotasDesaprobadasNoParcNoFin    0
ExcedeMitadCursosDesaprobados         0
ProgramaAlu                           0
Colegio                               0
TipoColegio                           0
Anio                                  0
Edad                                  0
GrupoEdad                             0
dtype: int64


In [19]:
print(df.shape)
df.to_csv('cachimbos_2023_2_sin_nulos.csv',index=False)

(1211, 20)


In [20]:
variables_modelo = ['Edad', 'Genero', 'Distrito', 'Provincia', 'Departamento', 'FamiliarResponsable', 'CantParcialesDesaprobados', 'CantFinalesDesaprobados','ExcedeNotasDesaprobadasNoParcNoFin', 'ExcedeMitadCursosDesaprobados','Desercion']
df_modelo = df[variables_modelo]

In [21]:
df_modelo = pd.get_dummies(df_modelo, columns=['Genero', 'Distrito', 'Provincia', 'Departamento', 'FamiliarResponsable', 'CantParcialesDesaprobados', 'CantFinalesDesaprobados','ExcedeNotasDesaprobadasNoParcNoFin', 'ExcedeMitadCursosDesaprobados'], drop_first=True)


# CREACION DEL MODELO

In [22]:
#División de los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X = df_modelo.drop(columns=['Desercion'])
y = df_modelo['Desercion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.astype(int)


In [23]:
# Diccionario que guarda las métricas
metricas_dict = {
    "Modelo": [],
    "Tipo_Data": [],
    "Accuracy": [],
    "AUC": [],
    "F1-Score": [],
    "Precisión": [],
    "Recall": [],
    "Puntuación": []
}

In [24]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score,f1_score, precision_score, recall_score
import xgboost as xgb
import shap

# Función para calcular métricas
def calcular_metricas(model, X_test, y_test, nombre_modelo):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)


    puntuacion = (0.3 * accuracy) + (0.1 * precision) + (0.3 * recall) + (0.3 * f1) + (0.5 * auc)
    
    metricas_dict["Modelo"].append(nombre_modelo)
    metricas_dict["Tipo_Data"].append("Data Original")
    metricas_dict["Accuracy"].append(accuracy)
    metricas_dict["AUC"].append(auc)
    metricas_dict["F1-Score"].append(f1)
    metricas_dict["Precisión"].append(precision)
    metricas_dict["Recall"].append(recall)
    metricas_dict["Puntuación"].append((puntuacion))


c:\Users\MSII7\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## MODELOS SIN BALANCEO

In [25]:
modelos = [
    ("Regresión Logística", LogisticRegression(max_iter=1000)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("XGBoost", xgb.XGBClassifier(eval_metric='logloss', random_state=42)),
    ("SVM", SVC(probability=True, random_state=42)),
    ("KNN",KNeighborsClassifier(n_neighbors=5))
]

In [26]:
# Entrenar y evaluar los modelos
for nombre_modelo, modelo in modelos:
    modelo.fit(X_train, y_train)
    calcular_metricas(modelo, X_test, y_test, nombre_modelo)


## MODELOS CON BALANCEO

In [27]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Reentrenar modelos con datos balanceados
for nombre_modelo, modelo in modelos:
    modelo.fit(X_resampled, y_resampled)
    calcular_metricas(modelo, X_test, y_test, f"{nombre_modelo} (SMOTE)")

## MODELOS CON BALANCEO Y AJUSTE DE HIPERPARAMETROS

In [28]:
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros para cada modelo
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear']
}

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

In [29]:
# Diccionario con modelos y sus respectivos grids
modelos_param_grids = [
    ("Regresión Logística", LogisticRegression(max_iter=1000), param_grid_lr),
    ("Random Forest", RandomForestClassifier(random_state=42), param_grid_rf),
    ("XGBoost", xgb.XGBClassifier(eval_metric='logloss', random_state=42), param_grid_xgb),
    ("SVM", SVC(probability=True, random_state=42), param_grid_svm),
    ("KNN", KNeighborsClassifier(), param_grid_knn)
]

In [30]:
# Realizamos la búsqueda de hiperparámetros
for nombre_modelo, modelo, param_grid in modelos_param_grids:
    grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
    grid_search.fit(X_resampled, y_resampled)
    
    # Entrenamos con los mejores hiperparámetros
    best_model = grid_search.best_estimator_
    print(f"Mejores parámetros para {nombre_modelo}: {grid_search.best_params_}")
    
    # Calculamos métricas con los mejores hiperparámetros
    calcular_metricas(best_model, X_test, y_test, f"{nombre_modelo} (Optimizado y con SMOTE)")

Mejores parámetros para Regresión Logística: {'C': 10, 'solver': 'liblinear'}
Mejores parámetros para Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 300}
Mejores parámetros para XGBoost: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 300}
Mejores parámetros para SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Mejores parámetros para KNN: {'n_neighbors': 7, 'weights': 'distance'}


# EXPORTADO DE METRICAS

In [31]:
df_metricas = pd.DataFrame(metricas_dict)

In [32]:
df_metricas = df_metricas.sort_values(by="Puntuación", ascending=False)
df_metricas.to_csv(os.path.join(ruta_output,'Modelo_Desercion_Metricas.csv'))
print(df_metricas)

                                          Modelo      Tipo_Data  Accuracy  \
13                  SVM (Optimizado y con SMOTE)  Data Original  0.741758   
2                                        XGBoost  Data Original  0.741758   
10  Regresión Logística (Optimizado y con SMOTE)  Data Original  0.725275   
12              XGBoost (Optimizado y con SMOTE)  Data Original  0.717033   
8                                    SVM (SMOTE)  Data Original  0.736264   
9                                    KNN (SMOTE)  Data Original  0.634615   
0                            Regresión Logística  Data Original  0.752747   
14                  KNN (Optimizado y con SMOTE)  Data Original  0.634615   
5                    Regresión Logística (SMOTE)  Data Original  0.711538   
7                                XGBoost (SMOTE)  Data Original  0.692308   
11        Random Forest (Optimizado y con SMOTE)  Data Original  0.678571   
1                                  Random Forest  Data Original  0.728022   